## X (Twitter) Post Generator Workflow

In [39]:
from langgraph.graph import StateGraph, START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict, Literal, Annotated
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage
import operator
import os
import re

In [40]:
load_dotenv()

True

In [41]:
# Get the Hugging Face token from the environment
hf_token = os.getenv("HUGGINGFACE_HUB_TOKEN")

# Check if token is loaded properly
if not hf_token:
    raise ValueError("HUGGINGFACE_HUB_TOKEN is not set in the environment!")

In [42]:
llm  = HuggingFaceEndpoint(
    repo_id="moonshotai/Kimi-K2-Instruct",
    task="text-generation",
    huggingfacehub_api_token=hf_token,
)







In [43]:

# Wrap in chat interface
chat_model = ChatHuggingFace(llm=llm)

In [44]:
# State definition
class TweetState(TypedDict):
    topic: str
    tweet: str
    evaluation: Literal["approved", "needs_improvement"]
    feedback: str
    iteration: int
    max_iteration: int
    tweet_history: Annotated[list[str], operator.add]
    feedback_history: Annotated[list[str], operator.add]

In [45]:

def generate_tweet(state: TweetState):
    """Generate initial tweet"""
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on: "{state['topic']}"

Rules:
- NO question-answer format
- Max 800 characters
- Use observational humor, irony, sarcasm
- Simple, everyday English
- Think meme logic and punchlines

Just return the tweet text, nothing else.
""")
    ]
    
    response = chat_model.invoke(messages).content.strip()
    print(f"Generated tweet: {response}")
    return {'tweet': response, 'tweet_history': [response]}

def evaluate_tweet(state: TweetState):
    """Evaluate tweet quality"""
    messages = [
        SystemMessage(content="You are a Twitter critic evaluating tweets for humor and virality."),
        HumanMessage(content=f"""
Evaluate this tweet: "{state['tweet']}"

Criteria:
1. Originality - Fresh or overused?
2. Humor - Actually funny?
3. Punchiness - Short and sharp?
4. Virality - Shareable?
5. Format - Well-formed tweet under 280 chars?

Auto-reject if:
- Question-answer format
- Over 600 characters
- Traditional setup-punchline
- Weak ending

Respond with ONLY:
evaluation: approved OR needs_improvement
feedback: [one paragraph explanation]
""")
    ]
    
    response = chat_model.invoke(messages).content.strip()
    
    # Parse response
    evaluation = "needs_improvement"  # default
    feedback = "Could not parse evaluation"
    
    if "approved" in response.lower():
        evaluation = "approved"
    
    # Extract feedback (everything after "feedback:")
    if "feedback:" in response.lower():
        feedback = response.split("feedback:")[-1].strip()
    else:
        feedback = response
    
    return {
        'evaluation': evaluation, 
        'feedback': feedback, 
        'feedback_history': [feedback]
    }

def optimize_tweet(state: TweetState):
    """Improve tweet based on feedback"""
    messages = [
        SystemMessage(content="You improve tweets for virality and humor."),
        HumanMessage(content=f"""
Improve this tweet based on feedback:

Topic: "{state['topic']}"
Current Tweet: "{state['tweet']}"
Feedback: "{state['feedback']}"

Write a better version. Avoid Q&A format, stay under 600 characters.
Return only the improved tweet.
""")
    ]
    
    response = chat_model.invoke(messages).content.strip()
    iteration = state['iteration'] + 1
    
    return {
        'tweet': response, 
        'iteration': iteration, 
        'tweet_history': [response]
    }

def route_evaluation(state: TweetState):
    """Route based on evaluation result"""
    if state['evaluation'] == 'approved' or state['iteration'] >= state['max_iteration']:
        return 'approved'
    return 'needs_improvement'



In [46]:

# Build workflow
graph = StateGraph(TweetState)

# Add nodes
graph.add_node('generate', generate_tweet)
graph.add_node('evaluate', evaluate_tweet)
graph.add_node('optimize', optimize_tweet)

# Add edges
graph.add_edge(START, 'generate')
graph.add_edge('generate', 'evaluate')
graph.add_conditional_edges(
    'evaluate', 
    route_evaluation, 
    {'approved': END, 'needs_improvement': 'optimize'}
)
graph.add_edge('optimize', 'evaluate')

# Compile workflow
workflow = graph.compile()



In [49]:
# Example usage
if __name__ == "__main__":
    initial_state = {
        "topic": "Jannik Sinner",
        "iteration": 1,
        "max_iteration": 3
    }
    
    result = workflow.invoke(initial_state)
    
    print("Generated Tweets:")
    for i, tweet in enumerate(result['tweet_history'], 1):
        print(f"{i}. {tweet}")
    
    print(f"\nFinal Status: {result['evaluation']}")
    print(f"Iterations: {result['iteration']}")

Generated tweet: Jannik Sinner’s forehand is so clean it does its own laundry, folds itself, and still has time to make the rest of the ATP look like they’re swinging a pool noodle.
Generated Tweets:
1. Jannik Sinner’s forehand is so clean it does its own laundry, folds itself, and still has time to make the rest of the ATP look like they’re swinging a pool noodle.

Final Status: approved
Iterations: 1
